In [13]:
import json
from langchain_openai import OpenAIEmbeddings
from functions import inference

In [2]:
with open("problem_set_g7.json", "r") as f:
    data = json.load(f)

In [3]:
n = len(data)
similar_score = [[0 for _ in range(n)] for _ in range(n)]
embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')

In [4]:
extract_prompt = r"""
    请从以下数学题目中提取关键信息。请返回以下格式的信息：

    1. 数学领域和类型（如代数、几何、微积分、统计等）
    2. 核心概念和原理（列出题目所需要的数学概念，如比例、相似、函数性质等）
    3. 解题方法和策略（列出用到特定的解题技巧，如配方法、换元法、归纳法等）
    4. 问题背景和情境（指出题目问题情境，如工程问题、经济问题、自然现象解释）
    5. 变量数量（题目所用到变量的数量，一定不要输出变量具体是什么）
    6. 题目所属年级（列出这道题最可能出现在哪一年级的试卷上）

    示例题目：
    求解方程 2x + 3 = 7

    示例输出：
    1. 数学领域和类型: 代数
    2. 核心概念和原理: 方程
    3. 解题方法和策略: 移项
    4. 问题背景和情境: 无
    5. 变量数量: 1
    6. 题目所属年级: 4

    现在请处理以下题目：
    {problem}
"""

In [5]:
problems = []
for item in data:
    problems.append(item['problem'])
len(problems)

103

In [7]:
import re
problems_emb = []
cache = {}
for problem in problems:
    attribs = []
    embedds = []
    temp = await inference({"problem": problem}, extract_prompt)
    pattern = r'\d+\.\s.*?:\s*(.*)'
    matches_p1 = re.findall(pattern, temp)
    for i, match in enumerate(matches_p1, 1):
        attribs.append(match)
    for att in attribs:
        if att in cache:
            embedds.append(cache.get(att))
        else:
            eb = embedding_model.embed_query(att)
            embedds.append(eb)
            cache[att] = eb
    problems_emb.append(embedds)

executing
1. 数学领域和类型: 几何
2. 核心概念和原理: 面积、正方形
3. 解题方法和策略: 开平方
4. 问题背景和情境: 学校活动
5. 变量数量: 1
6. 题目所属年级: 5executing
1. 数学领域和类型: 几何
2. 核心概念和原理: 面积、平方根
3. 解题方法和策略: 开平方
4. 问题背景和情境: 无
5. 变量数量: 1
6. 题目所属年级: 6executing
1. 数学领域和类型: 代数
2. 核心概念和原理: 算术平方根
3. 解题方法和策略: 平方根计算
4. 问题背景和情境: 无
5. 变量数量: 3
6. 题目所属年级: 6executing
1. 数学领域和类型: 数学基础
2. 核心概念和原理: 平方根、平方
3. 解题方法和策略: 直接计算
4. 问题背景和情境: 无
5. 变量数量: 6
6. 题目所属年级: 5executing
1. 数学领域和类型: 几何
2. 核心概念和原理: 面积、正方形
3. 解题方法和策略: 开平方
4. 问题背景和情境: 日常生活问题
5. 变量数量: 1
6. 题目所属年级: 5executing
1. 数学领域和类型: 代数
2. 核心概念和原理: 平方根
3. 解题方法和策略: 开平方
4. 问题背景和情境: 无
5. 变量数量: 1
6. 题目所属年级: 5executing
1. 数学领域和类型: 代数
2. 核心概念和原理: 平方根
3. 解题方法和策略: 直接计算
4. 问题背景和情境: 无
5. 变量数量: 4
6. 题目所属年级: 6executing
1. 数学领域和类型: 代数
2. 核心概念和原理: 平方根
3. 解题方法和策略: 定义验证
4. 问题背景和情境: 无
5. 变量数量: 7
6. 题目所属年级: 7executing
1. 数学领域和类型: 代数
2. 核心概念和原理: 平方根、算术平方根
3. 解题方法和策略: 概念辨析
4. 问题背景和情境: 无
5. 变量数量: 0
6. 题目所属年级: 7executing
1. 数学领域和类型: 代数
2. 核心概念和原理: 平方根、平方、正数和负数的性质
3. 解题方法和策略: 数学概念理解、反例验证
4. 问题背景和情境: 无
5. 变量数量: 1
6. 题目所属年级: 7exec

In [14]:
len(problems_emb)

103

In [9]:
with open("problems_emb.json", "w", encoding='utf-8') as f:
    json.dump(problems_emb, f, ensure_ascii=False, indent=4)

In [11]:
problem_q = "如果一个正方形的面积是36， 那么他的边长是多少"
q_emb = []
q_att = []
temp = await inference({"problem": problem_q}, extract_prompt)
pattern = r'\d+\.\s.*?:\s*(.*)'
matches_p1 = re.findall(pattern, temp)

for i, match in enumerate(matches_p1, 1):
    q_att.append(match)
    q_emb.append(embedding_model.embed_query(match))
q_emb

executing
1. 数学领域和类型: 几何
2. 核心概念和原理: 面积、正方形、平方根
3. 解题方法和策略: 开平方
4. 问题背景和情境: 无
5. 变量数量: 1
6. 题目所属年级: 5

[[-0.015129118226468563,
  0.00017279111489187926,
  0.0033321776427328587,
  0.029119638726115227,
  0.03034363128244877,
  0.004358694888651371,
  -0.008781435899436474,
  0.06267981231212616,
  -0.008333113044500351,
  -0.053912606090307236,
  -0.010510681197047234,
  0.0017070071771740913,
  0.014075915329158306,
  -0.024736037477850914,
  0.018046775832772255,
  0.01962658017873764,
  0.0014472645707428455,
  0.012560157105326653,
  -0.004650460556149483,
  -0.033674031496047974,
  0.002657914301380515,
  -0.013940706849098206,
  0.046340931206941605,
  0.047024089843034744,
  0.019413093104958534,
  0.008824133314192295,
  0.039651669561862946,
  0.0161253921687603,
  -0.03905390575528145,
  -0.012432064861059189,
  0.019939694553613663,
  0.030286701396107674,
  0.002542275469750166,
  0.004358694888651371,
  0.006216032430529594,
  0.04252662882208824,
  0.012332437559962273,
  0.011955277062952518,
  -0.008183672092854977,
  -0.042839743196964264,
  -0.021120989695191383,
  0.

In [15]:
import torch
q_emb = torch.tensor(q_emb)

In [17]:
problems_emb = torch.tensor(problems_emb)
problems_emb.shape

torch.Size([103, 6, 3072])

In [18]:
weights = [0.4, 0.2, 0.15, 0.05, 0.05, 0.15]
record = []
for i in range(problems_emb.shape[0]):
    saved_problem = problems_emb[i]
    similar_score = 0
    for j in range(q_emb.shape[0]):
        similar_score += torch.dot(q_emb[j], saved_problem[j]) * weights[j]
    record.append(similar_score)
record

[tensor(0.9338),
 tensor(0.9284),
 tensor(0.6070),
 tensor(0.6215),
 tensor(0.9350),
 tensor(0.7461),
 tensor(0.5949),
 tensor(0.5663),
 tensor(0.5597),
 tensor(0.5781),
 tensor(0.6116),
 tensor(0.8815),
 tensor(0.8340),
 tensor(0.5219),
 tensor(0.5853),
 tensor(0.5857),
 tensor(0.6011),
 tensor(0.7592),
 tensor(0.7827),
 tensor(0.8033),
 tensor(0.5768),
 tensor(0.5338),
 tensor(0.4297),
 tensor(0.5040),
 tensor(0.5551),
 tensor(0.5658),
 tensor(0.4944),
 tensor(0.5765),
 tensor(0.5768),
 tensor(0.5247),
 tensor(0.5501),
 tensor(0.5236),
 tensor(0.5273),
 tensor(0.5705),
 tensor(0.4909),
 tensor(0.4991),
 tensor(0.4909),
 tensor(0.5033),
 tensor(0.5094),
 tensor(0.4985),
 tensor(0.5471),
 tensor(0.4773),
 tensor(0.4648),
 tensor(0.4816),
 tensor(0.4663),
 tensor(0.4573),
 tensor(0.5750),
 tensor(0.6848),
 tensor(0.4780),
 tensor(0.4866),
 tensor(0.4722),
 tensor(0.5087),
 tensor(0.5177),
 tensor(0.5024),
 tensor(0.5015),
 tensor(0.5184),
 tensor(0.4970),
 tensor(0.5123),
 tensor(0.5093

In [22]:
record.index(max(record))

4

In [23]:
problems[4]

'小东想裁出一块面积为25平方分米的正方形画布，那么这块正方形画布的边长应取多少？'